In [141]:
import CadToFemModule as c
from OCC.BRepAlgoAPI import BRepAlgoAPI_Cut

#creating shape

pipe_plus=c.pipe_fillet(0.2)
pipe_minus=c.pipe_fillet(0.15)
#glued_pipe = BRepAlgoAPI_Cut(pipe_plus, pipe_minus).Shape()
glued_pipe = BRepAlgoAPI_Cut(pipe_plus, pipe_minus).Shape()
geometry = c.DisplayShapeFunc(glued_pipe)

#end of creating a shape

In [143]:
#diplaying shape
display(geometry)

# meshing shape. This mesh is used by the STL exporter
c.Mesh(glued_pipe)

# writing geometry to stl file and displaying geometry from stl file to check whether is correct 
stl_filename=c.stl_file("pipe", glued_pipe) 
print(stl_filename)

# creating geo from stl file
c.Geo_file(stl_filename)

# meshing using gmsh algo from geo
c.gmsh_algo(stl_filename)

./pipe_low_resolution.stl
b"Unexpected end of /proc/mounts line `overlay / overlay rw,relatime,lowerdir=/var/lib/docker/overlay2/l/I4DEZDWZP3OITBUYIPYK5MTASZ:/var/lib/docker/overlay2/l/DKNH3F7HEF4QXQM73EQDDT3L4H:/var/lib/docker/overlay2/l/OV4G27HFWH2PX3PJGTA6GWMOW2:/var/lib/docker/overlay2/l/A6AOAST2YG5IZ5MUGTZTYDE6KN:/var/lib/docker/overlay2/l/MNLBJVR65BTL6M2HMSB3EV4Z5M:/var/lib/docker/overlay2/l/XMQVTJAWSMBRID2B24OMQVQK55:/var/lib/docker/overlay2/l/AMO7HLXYA6IAMNFHG62WODX67G:/var/lib/docker/overlay2/l/Q455HM4KDEYCJBDKL2YHG3LIVT:/var/lib/docker/overlay2/l/HMWFDYXGUA5CT'\nUnexpected end of /proc/mounts line `ULJ253LV5GIOC:/var/lib/docker/overlay2/l/DQZCAIYKE6UY7J67V6DMW2347I:/var/lib/docker/overlay2/l/FW3NFIKILVSUSWKXXO4XKIGB3R:/var/lib/docker/overlay2/l/RWNYIECQOKTLQZZKCPE2DNS5CA:/var/lib/docker/overlay2/l/VJHELC76SPK4OUW4DFBSLV6OUF:/var/lib/docker/overlay2/l/ZHZOTJDGWYASRZBSOS776BXEBD:/var/lib/docker/overlay2/l/WMK4MK6KMDP2UQMBEYWE7YPZIQ:/var/lib/docker/overlay2/l/PJDASAZRCISOZVN7KO3

In [144]:
from dolfin import *
from IPython.display import display, HTML
mesh = Mesh("./tmp/sfm.xml")
V = FunctionSpace(mesh, "CG", 1)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0.1)
a = dot(grad(u), grad(v))*dx
L = f*v*dx

# Define boundary condition values
u0 = Constant(0.0)
u1 = Constant(1.0)
u2 = Constant(2.0)
u3 = Constant(3.0)

# Define boundary conditions
bc0 = DirichletBC(V, u0, 0)
bc1 = DirichletBC(V, u1, 1)
bc2 = DirichletBC(V, u2, 2)
bc3 = DirichletBC(V, u3, 3)

# Set PETSc MUMPS paramter (this is required to prevent a memory error
# in some cases when using MUMPS LU solver).
if has_petsc():
    PETScOptions.set("mat_mumps_icntl_14", 40.0)

# Compute solution
u = Function(V)
solve(a == L, u, [bc0, bc1, bc2, bc3])

# Write solution to file
File("tmp/u.pvd") << u
HTML(X3DOM().html(u))